In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## Read Data

In [2]:
df = pd.read_csv('mapping.csv')
df = df.drop(['cosine_name', 'cosine_address', 'distance', 'id',
              'created_date_id', 'created_datetime', 'rank_point'], axis=1)
df.head()

,hotel_id,domain_id,domain_hotel_id,similar_point
0,1,2,9000000337180,0.528068
1,2,2,1000000503984,0.671261
2,4,2,1000000430294,0.857708
3,5,2,1000000432030,0.862712
4,6,2,3000010027516,0.766834


In [8]:
roothotel_info = pd.read_csv('table/roothotel_info.csv')
roothotel_info.head(2)

,id,name,address,logo,province_id,district_id,street_id,latitude,longitude,star_number,overall_score,checkin_time,checkout_time,description,created_date_id,created_datetime
0,1,OLIVE - Khách sạn Demo,"123231 Bắc Sơn, Quận Ba Đình, Hà Nội",https://www.googleapis.com/download/storage/v1...,11,15.0,28756.0,21.032986,105.839120,3,60.0,00:00,12:00,<p>Khách sạn Demo&nbsp;</p>\n,20200713,2020-07-13 02:21:53
1,2,Khách sạn Nhật Linh,"194 Nguyễn Chí Thanh, Quận Hải Châu, Đà Nẵng",https://www.googleapis.com/download/storage/v1...,50,238.0,26881.0,16.064370,108.220542,3,84.0,14:00,12:00,<p style=text-align: justify;>Tọa lạc tại vị t...,20200713,2020-07-13 02:21:53


In [10]:
ota_cleaned = ota[['ota_name', 'domain_hotel_id', 'domain_id']]
ota_cleaned.head()

,ota_name,domain_hotel_id,domain_id
0,Roha Hotel,9000000760259,2
1,Co Home - Hostel,9000000797054,2
2,Hoang Su Phi Lodge,1000001040447,2
3,Sao Viet Hotel,3000010037920,2
4,Van Apartment,3000020017271,2


## Clean Data

1. cosine_hotel

In [3]:
df_cleaned = df.groupby(['domain_hotel_id']).agg(np.max)

In [4]:
df_cleaned = df_cleaned.sort_values(['similar_point'], ascending=0)

In [5]:
df_cleaned.reset_index(inplace=True)

In [6]:
df_cleaned = df_cleaned.head(7000)

In [7]:
# df_cleaned.to_csv('df_cleaned.csv', index=False)

2. hotel_v4_distinct

In [9]:
ota = pd.read_csv('table/hotel_v4_distinct.csv')
ota = ota.rename(columns={'name': 'ota_name'})
ota.head()

,domain_hotel_id,domain_id,ota_name,url,city_id,crawled_at,crawled_date_id,crawled_datetime
0,9000000760259,2,Roha Hotel,9000000760259,23,1597202348,20200812,2020-08-12 03:19:08
1,9000000797054,2,Co Home - Hostel,9000000797054,29,1597202328,20200812,2020-08-12 03:18:48
2,1000001040447,2,Hoang Su Phi Lodge,1000001040447,29,1597202328,20200812,2020-08-12 03:18:48
3,3000010037920,2,Sao Viet Hotel,3000010037920,6,1597202324,20200812,2020-08-12 03:18:44
4,3000020017271,2,Van Apartment,3000020017271,4,1597202250,20200812,2020-08-12 03:17:30


In [14]:
ota_cleaned['domain_hotel_id'].value_counts()

6948697     5
16260062    5
5926435     5
5247777     5
5006283     5
           ..
5603930     1
5872217     1
4844119     1
4454997     1
788481      1
Name: domain_hotel_id, Length: 53916, dtype: int64

3. roothotel_info

In [11]:
hotel_name = roothotel_info[['id' ,'name']]
hotel_name = hotel_name.rename(columns={'id': 'hotel_id', 'name': 'tripi_name'})
hotel_name.head()

,hotel_id,tripi_name
0,1,OLIVE - Khách sạn Demo
1,2,Khách sạn Nhật Linh
2,4,Sunset Westlake Hanoi Hotel
3,5,"Khách sạn Công Đoàn Thanh Bình, Đà Nẵng (Cong ..."
4,6,Khách sạn Lavender Nha Trang (Lavender Nha Tra...


## Merge

In [12]:
df_grouped = df_cleaned.merge(hotel_name, on='hotel_id')
df_grouped = df_grouped.merge(ota_cleaned, on='domain_hotel_id')

In [13]:
df_grouped = df_grouped[['hotel_id', 'tripi_name', 'ota_name',
                        'similar_point', 'domain_hotel_id', 'domain_id_x' , 'domain_id_y']]
df_grouped.sort_values(['hotel_id'])

,hotel_id,tripi_name,ota_name,similar_point,domain_hotel_id,domain_id_x,domain_id_y
1193,32,Vinpearl Resort Nha Trang,Vinpearl Resort Nha Trang,1.000000,65153,3,3
1194,32,Vinpearl Resort Nha Trang,Vinpearl Resort Nha Trang,1.000000,65153,3,3
1195,32,Vinpearl Resort Nha Trang,Vinpearl Resort Nha Trang,1.000000,65153,3,3
7494,67,Khách sạn Sunway Hà Nội (Sunway Hotel),Sunway Hotel,0.969607,10967,3,3
4504,70,Villa Del Sol Beach Resort & Spa,Villa Del Sol Beach Resort & Spa,1.000000,296617,3,3
...,...,...,...,...,...,...,...
2997,47675,TP Beachfront Apartment,Handy Beachfront Apartment,1.000000,3149971,3,3
2998,47675,TP Beachfront Apartment,Handy Beachfront Apartment,0.992004,2765491,5,5
6582,47676,Nina Apartments,Nina Apartments,0.981075,45336009,4,4
6583,47676,Nina Apartments,Nina Apartments,0.971737,9000000728268,2,2


In [16]:
df_grouped.sort_values(['hotel_id'])

,hotel_id,tripi_name,ota_name,similar_point,domain_hotel_id,domain_id_x,domain_id_y
1193,32,Vinpearl Resort Nha Trang,Vinpearl Resort Nha Trang,1.000000,65153,3,3
1194,32,Vinpearl Resort Nha Trang,Vinpearl Resort Nha Trang,1.000000,65153,3,3
1195,32,Vinpearl Resort Nha Trang,Vinpearl Resort Nha Trang,1.000000,65153,3,3
7494,67,Khách sạn Sunway Hà Nội (Sunway Hotel),Sunway Hotel,0.969607,10967,3,3
4504,70,Villa Del Sol Beach Resort & Spa,Villa Del Sol Beach Resort & Spa,1.000000,296617,3,3
...,...,...,...,...,...,...,...
2997,47675,TP Beachfront Apartment,Handy Beachfront Apartment,1.000000,3149971,3,3
2998,47675,TP Beachfront Apartment,Handy Beachfront Apartment,0.992004,2765491,5,5
6582,47676,Nina Apartments,Nina Apartments,0.981075,45336009,4,4
6583,47676,Nina Apartments,Nina Apartments,0.971737,9000000728268,2,2


* Check wrong

In [17]:
df_grouped['domain_id_x'].value_counts()

3    5091
5    1781
4     638
2     254
Name: domain_id_x, dtype: int64

In [18]:
df_grouped['domain_id_y'].value_counts()

3    5095
5    1777
4     638
2     254
Name: domain_id_y, dtype: int64

In [27]:
temp = df_grouped[df_grouped['domain_id_x'] == 5]
temp = temp[temp['domain_id_y'] == 3]
temp

,hotel_id,tripi_name,ota_name,similar_point,domain_hotel_id,domain_id_x,domain_id_y
35,47183,Khách sạn Hà Vân,Golden Summer Hotel Nha Trang,1.000000,271323,5,3
1575,40932,Mama Chai Homestay,City Poshtel,1.000000,5462189,5,3
2412,32629,YHD Hotel,Adamas Apartment,0.988173,2302897,5,3
3299,44805,Diep Homestay,Diep Homestay,1.000000,6382532,5,3
4106,30568,Hoang Long Son 2 Hotel,Hoang Long Son 2 Hotel,1.000000,283849,5,3
6915,46002,Hoang Hop Homestay - Hostel,Ba Be Hoang Hop Homestay,0.976511,4207888,5,3


In [28]:
df_grouped.to_csv('df_grouped.csv', index=False)